# Using this notebook one can convert data arrays from SED models to suitable files 1D-Spectra uselful for the MOONS-ETC

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PyAstronomy import pyasl
from astropy.table import Table
from astropy.io import fits
# import spectres
import warnings
import os
from pysynphot import observation
from pysynphot import spectrum

/Users/lam/anaconda3/lib/python3.7/site-packages/pysynphot/locations.py:47: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/Users/lam/anaconda3/lib/python3.7/site-packages/pysynphot/locations.py:346: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


# Resampling BC03 models

In [3]:
# Data to create the names

data          = Table.read('/Users/lam/Desktop/MOONS/MOONS_2021/COSMOS_FMOS_MOONS_2021_zCUT_BPT.fits', format='fits')
data          = data[:-1]

galaxy_names = data['id']

In [4]:
def Lobs(Lambda,redshift):
    return (Lambda*(redshift + 1.0))

def flux_lambda(flux_nu, wavelength):
    # Wavelength must be in nm
    # Flux_nu must be in mJy
# #     return((1.0/3.34e4)*(wavelength*10)**-2*(flux_nu*1e-3))
    # Wavelength must be in A
    # Flux_nu must be in mJy
    return((1.0/3.34e4)*(wavelength)**-2*(flux_nu*1e-3))

In [5]:
# Deleting the folders if already created

os.system('rm -rf /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/')
os.system('rm -rf /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/')

os.system('rm -rf /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/uniform/')
os.system('rm -rf /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/uniform_fluxConserved/')

os.system('rm -rf /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/uniform/')
os.system('rm -rf /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/uniform_fluxConserved/')

# Creating the folders

os.system('mkdir /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/')
os.system('mkdir /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/')

os.system('mkdir /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/uniform/')
os.system('mkdir /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/uniform_fluxConserved/')

os.system('mkdir /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/uniform/')
os.system('mkdir /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/uniform_fluxConserved/')

# For the ETC final results
os.system('rm -rf /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/ETC_format/')
os.system('rm -rf /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/ETC_format/')

os.system('mkdir /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/ETC_format/')
os.system('mkdir /Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/ETC_format/')


0

# Simple linear interpoaltion because BC03 are non-uniform distributed

In [ ]:
# the new wavelength range vector is created
# I chose this because if I create a vector with 0.9AA there are too many values
# The vector cannot be created due to a MemoryError

path = '/Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/'
# path = '/Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/'

path_data = '/Users/lam/Desktop/MOONS/MOONS_2021/metal_bins/continuum_lines/Salpeter/All_bins/'
# path_data = '/Users/lam/Desktop/MOONS/MOONS_2021/metal_bins/continuum_lines/Chabrier/All_bins/'

for i in range(len(galaxy_names)):
    
    data_BC03 = Table.read(path_data+str(galaxy_names[i])+'_best_model.fits', format = 'fits') 
    
    # Wavelength from nm to Angstrom I multiply by 10
    data_BC03['wavelength'] = 10*data_BC03['wavelength']
        
#         I stopped in 100000A because I consider is enough in wavelength
#         The spacing for the sample according to MOONS resolution is 0.23AA-0.35AA for (8000-18000 and 0.72-1.6AA)
    
    
    data_BC03 = data_BC03[data_BC03['wavelength'] <= 110000]
    
    a = np.min(data_BC03['wavelength'])
    wvl_vector = [a]
    while a < np.max(data_BC03['wavelength']):
        a = a + 0.23
        if a > 100000:
            break
        wvl_vector.append(a)  
        
    new_spec_wavs = np.array(wvl_vector)
    old_spec_wavs = data_BC03['wavelength']
    # Flux from mJy to erg cm^-2 s^-1 A^-1    
    spec_fluxes   = flux_lambda(data_BC03['Fnu'], data_BC03['wavelength'])         
        
#         New fluxes are created by inteporlating the new wavelength grid and the old data set
#         res_fluxes = spectres.spectres(new_spec_wavs, old_spec_wavs, spec_fluxes, spec_errs=None)
    res_fluxes = np.interp(new_spec_wavs, old_spec_wavs, spec_fluxes)
        
    model_BC03 = Table([new_spec_wavs, res_fluxes], names=('wavelength', 'flux'),\
                        dtype=('f8', 'f8'), meta={'name': str(galaxy_names[i])+'_BC03_'+'uniform'})
        
    model_BC03.write(path+'uniform/ID_'+str(galaxy_names[i])+'_BC03_continuum_lines.fits',\
                     format = 'fits', overwrite = True)


# Linear interpoaltion because BC03 are non-uniform distributed (conserves flux)

In [ ]:
def rebin_spec(wave, specin, wavnew):
    spec = spectrum.ArraySourceSpectrum(wave=wave, flux=specin)
    f = np.ones(len(wave))
    filt = spectrum.ArraySpectralElement(wave, f, waveunits='angstrom')
    obs = observation.Observation(spec, filt, binset=wavnew, force='taper')
 
    return(obs.binflux)

In [ ]:
# Wavelengths are created only once outside
# The 0.23AA comes from the MOONS resolution so we resample everything
# to be consistent with MOONS and also CIGALE and BC03 models

# BC03 wavlength limits
# 91.0-100000000.0

path = '/Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/'
# path = '/Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/'

path_data = '/Users/lam/Desktop/MOONS/MOONS_2021/metal_bins/continuum_lines/Salpeter/All_bins/'
# path_data = '/Users/lam/Desktop/MOONS/MOONS_2021/metal_bins/continuum_lines/Chabrier/All_bins/'

# l = np.min(data_BC03['wavelength'])
l = 91.0
wvl_vector = [l]
# while l < np.max(data_BC03['wavelength']):
while l < 100000000.0:
    l = l + 0.23
    if l > 100000:
        break
    wvl_vector.append(l)  
wvl_vector = np.array(wvl_vector)

new_spec_wavs = np.array(wvl_vector)

# interpolation per galaxy
for i in range(len(galaxy_names)):
    
    data_BC03 = Table.read(path_data+str(galaxy_names[i])+'_best_model.fits', format = 'fits')
    
    # Wavelength from nm to Angstrom I multiply by 10
    data_BC03['wavelength'] = 10*data_BC03['wavelength']    
        
#         I stopped in 100000A because I consider is enough in wavelength
#         The spacing for the sample according to BC03 is 0.9AA

    data_BC03 = data_BC03[data_BC03['wavelength'] <= 110000]

    old_spec_wavs = data_BC03['wavelength']
    # Flux from mJy to erg cm^-2 s^-1 A^-1    
    spec_fluxes   = flux_lambda(data_BC03['Fnu'], data_BC03['wavelength']) 
        
#         New fluxes are created by inteporlating the new wavelength grid and the old data set
    spec       = spectrum.ArraySourceSpectrum(wave=old_spec_wavs, flux=spec_fluxes)
    f          = np.ones(len(old_spec_wavs))
    filt       = spectrum.ArraySpectralElement(old_spec_wavs, f, waveunits='angstrom')
    obs        = observation.Observation(spec, filt, binset=new_spec_wavs, force='taper')
    res_fluxes = obs.binflux
        
    # Units for flux are in erg cm^-2 s^-1 A^-1        
    model_BC03 = Table([new_spec_wavs, res_fluxes], names=('wavelength', 'flux'),\
                        dtype=('f8', 'f8'), meta={'name': str(galaxy_names[i])+'_BC03_'+'uniform_fluxConserved'})
        
    model_BC03.write(path+'uniform_fluxConserved/ID_'+str(galaxy_names[i])+'_BC03_continuum_lines.fits',\
                     format = 'fits', overwrite = True)        

# Creating the new files in the right format for the ETC
# Interpolation with flux conservation

In [ ]:
# This cannot be used for BC03 models directly because the UV-part and the Optical have non-uniform spacing

path = '/Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Salpeter_continuum_lines/'
# path = '/Users/lam/Desktop/MOONS/MOONS_2021/Spectra_Generation_2021/Chabrier_continuum_lines/'

path_data = '/Users/lam/Desktop/MOONS/MOONS_2021/metal_bins/continuum_lines/Salpeter/All_bins/'
# path_data = '/Users/lam/Desktop/MOONS/MOONS_2021/metal_bins/continuum_lines/Chabrier/All_bins/'

for i in range(len(galaxy_names)):
        
    data_BC03 = Table.read(path+'uniform_fluxConserved/ID_'+str(galaxy_names[i])+'_BC03_continuum_lines.fits', format = 'fits')
            
            
# R should be equal to central_wavelength (A) / (delta_lambda in Ang) = [(6800.2 + 1000.2) / 2.]/0.55 = 7091

# Sampling is pixels – Over how many pixels the element of resolution is spead. Usually 2 or 3 
# (to be Nyquist sampled) I guess is 0.55/0.2 = 2.75 in your case.


    header_BC03 = {"CODE":'CIGALE / Simulation Code',\
                   "MTYPE":'Galaxy',\
                   "MNAME": str(galaxy_names[i])+' / Model Name',\
                   "R":2200.0,\
                   "SAMPLING":0.23,\
                   "V_disp":0.0,\
                   "TUNIT1":'Angstroms',\
                   "TUNIT2":'erg/s/cm2/A',\
                   "LAMBDA0":5445.00}        
        
    # Passing wavelength directly
                
    wvl_BC03 = data_BC03['wavelength']

    pyasl.write1dFitsSpec(path+'ETC_format/ID_'+str(galaxy_names[i])+'_BC03_continuum_lines.fits', data_BC03['flux'], wvl=wvl_BC03, header=header_BC03, clobber=True)       
    